## Data pipeline

All later components (baseline, RAG, metrics) need the same corpus and review “ground truth.” Cleaning and structuring it early prevents painful re‑work.

• Pick a public peer‑review dataset (e.g., PeerRead, ACL Blind Submission). <br>
• Write a loader that yields (paper_text, meta, gold_review) triples. <br>
• Basic preprocessing: strip markup, split into sections, tokenize. <br>


In [1]:
print("Hello world")

Hello world


### EDA: Review_mt dataset (not useful for now)

Each json file contains a list of dictionaries with the respective entries:

[
  {
    "instruction": "...",
    "input": "..."
  },
  {
    "instruction": "...",
    "input": "..."
  },
  ...
]

In [ ]:
import json

def load_reviewmt_file_train():
    with open("data/reviewmt_train/00.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

# Example usage
data = load_reviewmt_file_train()

example_train = data[0]
print("Instruction:", example_train["instruction"])
print("Input:", example_train["input"])

example = data[100]
print("Instruction:", example_train["instruction"])
print("Input:", example_train["input"])

Instruction: You areonses from the author and comments from all reviewers to provide the meta-review and determine the final decision. E a decision maker. Please review all respxplicitly state 'Accept' or 'Reject' at the end of your output.
Input: Title: Characterizing and Measuring the Similarity of Neural Networks with Persistent Homology . Abstract: Characterizing the structural properties of neural networks is crucial yet poorly understood, and there are no well-established similarity measures between networks. In this work, we observe that neural networks can be represented as abstract simplicial complex and analyzed using their topological 'fingerprints' via Persistent Homology (PH). We then describe a PH-based representation proposed for characterizing and measuring similarity of neural networks. We empirically show the effectiveness of this representation as a descriptor of different architectures in several datasets. This approach based on Topological Data Analysis is a step t

In [27]:
print("Test", example_train["Summary"])

KeyError: 'Summary'

In [24]:
# Number of examples
print("Number of examples:", len(data))

Number of examples: 2000


In [25]:
def load_reviewmt_file_test():
    with open("data/reviewmt_test.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

test_data= load_reviewmt_file_test()

example_test = test_data[0]
print("Instruction:", example_test["instruction"])
print("Input:", example_test["input"])

Instruction: You are a decision maker. Please review all responses from the author and comments from all reviewers to provide the meta-review and determine the final decision. Explicitly state 'Accept' or 'Reject' at the end of your output.
Input: Title: The Infinite Contextual Graph Markov Model. Abstract: The Contextual Graph Markov Model is a deep, unsupervised, and probabilistic model for graphs that is trained incrementally on a layer-by-layer basis. As with most Deep Graph Networks, an inherent limitation is the lack of an automatic mechanism to choose the size of each layer's latent representation. In this paper, we circumvent the problem by extending the Contextual Graph Markov Model with Hierarchical Dirichlet Processes. The resulting model for graphs can automatically adjust the complexity of each layer without the need to perform an extensive model selection. To improve the scalability of the method, we introduce a novel approximated inference procedure that better deals wit

## 

### EDA: ICLR 2017 dataset (promising)

#### I still dont really understand what these dataset contains, it look like it only has the LLM instruction plus the full paper title, where is the rest?? -> Look at the other dataset on GitHub

In [7]:
def load_iclr_file():
    with open("data/ICLR_2017.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

test_data = load_iclr_file()

print(test_data[0].keys())

example_test = test_data[0]

dict_keys(['instruction', 'input', 'output', 'system', 'history'])


In [45]:
import re

def parse_input_sections(text: str) -> dict:
    pattern = r"Title:\s*(.*?)\s*Abstract:\s*(.*?)\s*Main Text:\s*(.*)"
    match = re.match(pattern, text, re.DOTALL)
    if match:
        return {
            "title": match.group(1).strip(),
            "abstract": match.group(2).strip(),
            "main_text": match.group(3).strip()
        }
    else:
        raise ValueError("Input text does not match expected pattern.")

def parse_review_sections(text: str) -> dict:
    """
    Parse a review into 'Summary', 'Strengths and Weaknesses', and optionally 'Questions'.
    """
    pattern = (
        r"Summary:\s*(.*?)\s*"
        r"Strengths and Weaknesses:\s*(.*?)(?:\s*Questions:\s*(.*))?$"
    )
    match = re.match(pattern, text, re.DOTALL)
    if not match:
        raise ValueError("Review does not match expected format.")
    
    result = {
        "summary": match.group(1).strip(),
        "strengths_and_weaknesses": match.group(2).strip(),
        "questions": match.group(3).strip() if match.group(3) else None
    }
    return result

In [8]:
print("Instruction:", example_test["instruction"])

Instruction: Role: Decision Maker. Task: Review all responses from the author and comments from all reviewers to provide the meta-review and determine the final decision. Explicitly state 'Accept' or 'Reject' at the end of your output.


In [9]:
print("Input:", example_test["input"])

Input: Title: Identity Matters in Deep Learning Abstract: An emerging design principle in deep learning is that each layer of a deep
artificial neural network should be able to easily express the identity
transformation. This idea not only motivated various normalization techniques,
such as batch normalization, but was also key to the immense success of
residual networks.

In this work, we put the principle of identity parameterization on a more 
solid theoretical footing alongside further empirical progress. We first
give a strikingly simple proof that arbitrarily deep linear residual networks
have no spurious local optima. The same result for feed-forward networks in
their standard parameterization is substantially more delicate.  Second, we
show that residual networks with ReLu activations have universal finite-sample
expressivity in the sense that the network can represent any function of its
sample provided that the model has more parameters than the sample size.

Directly inspire

In [33]:
sections = parse_input_sections(example_train["input"])
print("Title:\n", sections["title"])
print("\nAbstract:\n", sections["abstract"])
print("\nMain Text:\n", sections["main_text"][:500], "...")

Title:
 Characterizing and Measuring the Similarity of Neural Networks with Persistent Homology .

Abstract:
 Characterizing the structural properties of neural networks is crucial yet poorly understood, and there are no well-established similarity measures between networks. In this work, we observe that neural networks can be represented as abstract simplicial complex and analyzed using their topological 'fingerprints' via Persistent Homology (PH). We then describe a PH-based representation proposed for characterizing and measuring similarity of neural networks. We empirically show the effectiveness of this representation as a descriptor of different architectures in several datasets. This approach based on Topological Data Analysis is a step towards better understanding neural networks and a useful similarity measure.

Main Text:
 Introduction

Machine learning practitioners can train different neural networks for the same task. Even for the same neural architecture, there are many h

In [10]:
print("Output:", example_test["output"])

Output: The paper begins by presenting a simple analysis for deep linear networks. This is more to demonstrate the intuitions behind their derivations, and does not have practical relevance. They then extend to non-linear resnets with ReLU units and demonstrate that they have finite sample expressivity. They formally establish these results. Inspired by their theory, they perform experiments using simpler architectures without any batch norm, dropout or other regularizations and fix the last layer and still attain competitive results. Indeed, they admit that data augmentation is a form of regularization and can replace other regularization schemes. 
 I think the paper meets the threshold to be accepted. Final decision: Accept (Poster)


In [11]:
print("System:", example_test["system"])

System: 


In [12]:
print("History:", example_test["history"])

History: [["Role: Reviewer 1. Style: Strict. Task: Provide a critical review based on the paper provided, including a summary, strengths, weaknesses, and any questions you have. Please provide your review of 'Identity Matters in Deep Learning'", "Summary: Paper Summary:\n\nAuthors investigate identity re-parametrization in the linear and the non linear case. \n\nDetailed comments:\n\n— Linear Residual Network:\n\nThe paper shows that for a linear residual network any critical point is a global optimum. This problem is non convex it is interesting that this simple re-parametrization leads to such a result. \n\n — Non linear Residual Network:\n\nAuthors propose a construction that maps the points to their labels via a resnet , using an initial random projection, followed by a residual block that clusters the data based on their label, and a last layer that maps the clusters to the label. \n\n1- In Eq 3.4  seems the dimensions are not matching q_j in R^k and e_j in R^r. please clarify \n\

In [14]:
print(len(example_test["history"]))

9


In [13]:
print(example_test["history"][0])

["Role: Reviewer 1. Style: Strict. Task: Provide a critical review based on the paper provided, including a summary, strengths, weaknesses, and any questions you have. Please provide your review of 'Identity Matters in Deep Learning'", "Summary: Paper Summary:\n\nAuthors investigate identity re-parametrization in the linear and the non linear case. \n\nDetailed comments:\n\n— Linear Residual Network:\n\nThe paper shows that for a linear residual network any critical point is a global optimum. This problem is non convex it is interesting that this simple re-parametrization leads to such a result. \n\n — Non linear Residual Network:\n\nAuthors propose a construction that maps the points to their labels via a resnet , using an initial random projection, followed by a residual block that clusters the data based on their label, and a last layer that maps the clusters to the label. \n\n1- In Eq 3.4  seems the dimensions are not matching q_j in R^k and e_j in R^r. please clarify \n\n2- The co

In [41]:
print(example_test["history"][0][1])

Summary: The paper combines the idea from the classical hidden dirichlet processe and CGMM to automate the process of selecting the hyperparameters of CGMM.
Interesting idea but not very impressive experimental results. Strengths and Weaknesses: The authors propose an interesting idea of utilizing the HDP framework to obtain the number of latent variables at each layer. HOwever the gain in the accuracy seems either non-existent or marginal.


In [46]:
review_text = example_test["history"][0][1]
sections = parse_review_sections(review_text)

print("Summary:\n", sections["summary"])
print("\nStrengths and Weaknesses:\n", sections["strengths_and_weaknesses"])
if sections["questions"]:
    print("\nQuestions:\n", sections["questions"])
else:
    print("\nQuestions: Not provided")

Summary:
 The paper combines the idea from the classical hidden dirichlet processe and CGMM to automate the process of selecting the hyperparameters of CGMM.
Interesting idea but not very impressive experimental results.

Strengths and Weaknesses:
 The authors propose an interesting idea of utilizing the HDP framework to obtain the number of latent variables at each layer. HOwever the gain in the accuracy seems either non-existent or marginal.

Questions: Not provided


In [13]:
print(example_test["history"][1])

["Role: Author. Task: Respond to Reviewer 1's comments by clarifying the mentioned weaknesses and answering the posed questions. Summarize the Reviewer 1's comment above and provide a response accordingly.", "Author does't have any rebuttal."]


In [15]:
print(example_test["history"][2])

["Role: Reviewer 1. Style: Strict. Task: Based on the author's response, provide a final score from 1 to 10 and a confidence from 1 to 5.", "Reviewer1 does't have any response to the Author. Score: 5: Marginally below acceptance threshold Confidence: 4: The reviewer is confident but not absolutely certain that the evaluation is correct"]


In [16]:
print(example_test["history"][3])

["Role: Reviewer 1. Style: Toletant. Task: Provide a critical review based on the paper provided, including a summary, strengths, weaknesses, and any questions you have. Please provide your review of 'Identity Matters in Deep Learning'", "Summary: This paper investigates the identity parametrization also known as shortcuts where the output of each layer has the form h(x)+x instead of h(x). This has been shown to perform well in practice (eg. ResNet). The discussions and experiments in the paper are interesting. Here's a few comments on the paper:\n\n-Section 2: Studying the linear networks is interesting by itself. However, it is not clear that how this could translate to any insight about non-linear networks. For example, you have proved that every critical point is global minimum. I think it is helpful to add some discussion about the relationship between linear and non-linear networks.\n\n-Section 3: The construction is interesting but the expressive power of residual network is wit

In [17]:
print(example_test["history"][4])

["Role: Author. Task: Respond to Reviewer 1's comments by clarifying the mentioned weaknesses and answering the posed questions. Summarize the Reviewer 1's comment above and provide a response accordingly.", "Author does't have any rebuttal."]


In [18]:
print(example_test["history"][5])

["Role: Reviewer 1. Style: Toletant. Task: Based on the author's response, provide a final score from 1 to 10 and a confidence from 1 to 5.", "Reviewer1 does't have any response to the Author. Score: 6: Marginally above acceptance threshold Confidence: 5: The reviewer is absolutely certain that the evaluation is correct and very familiar with the relevant literature"]


In [19]:
print(example_test["history"][6])

["Role: Reviewer 1. Style: Toletant. Task: Provide a critical review based on the paper provided, including a summary, strengths, weaknesses, and any questions you have. Please provide your review of 'Identity Matters in Deep Learning'", 'Summary: This paper provides some theoretical guarantees for the identity parameterization by showing that 1) arbitrarily deep linear residual networks have no spurious local optima; and 2) residual networks with ReLu activations have universal finite-sample expressivity. This paper is well written and studied a fundamental problem in deep neural network. I am very positive on this paper overall and feel that this result is quite significant by essentially showing the stability of auto-encoder, given the fact that it is hard to provide concrete theoretical guarantees for deep neural networks.\n\nOne of key questions is how to extent the result in this paper to the more general nonlinear actuation function case. \n\nMinors: one line before Eq. (3.1), U

In [20]:
print(example_test["history"][7])

["Role: Author. Task: Respond to Reviewer 1's comments by clarifying the mentioned weaknesses and answering the posed questions. Summarize the Reviewer 1's comment above and provide a response accordingly.", "Author does't have any rebuttal."]


In [21]:
print(example_test["history"][8])

["Role: Reviewer 1. Style: Toletant. Task: Based on the author's response, provide a final score from 1 to 10 and a confidence from 1 to 5.", "Reviewer1 does't have any response to the Author. Score: 8: Top 50% of accepted papers, clear accept Confidence: 3: The reviewer is fairly confident that the evaluation is correct"]


## Baseline LLM only

Gives you a running system in <100 lines, a sanity‑check for your metric code, and a performance floor you’ll later try to beat.

• Draft a prompt template that mirrors real reviews (summary, strengths, weaknesses, score). <br>
• CLI script: generate_review.py --model gpt-4o --paper_id X → JSON review. <br>
• Serialize outputs for 50–100 papers. <br>


## Evaluation harness

You want tight feedback loops. Implementing metrics now lets you plug in future variants with one line of code.


• Automatic overlap metrics (BERTScore, ROUGE). <br>
• Placeholder hooks for factual‑consistency checks (e.g., using GPT‑4 to judge). <br>
• Skeleton for a human‑rating spreadsheet or web form. <br>


## RAG plus Vector DB

Only once data is fixed can you build a stable RAG back‑end.

• Decide on chunk size & overlap; embed with an off‑the‑shelf model (e.g., Instructor or OpenAI Ada‑002). <br>
• Build a vector index (FAISS/Qdrant) and a lightweight search API. <br>


## Retrival powered generation

Now you can focus on retrieval + fusion strategies and immediately compare against the baseline using the harness you already trust.

• Retrieve top‑k chunks, concatenate with the prompt, generate review. <br>
• Experiment with reranking or citation‑insertion heuristics. <br>

